In [65]:
import pandas as pd
from glob import glob
import shutil
import matplotlib.pyplot as plt
import os

In [66]:
pre_file_list=glob('../../data/row_collection/이영섭/*gil.csv')
pre_move_file_list=[f.replace('/row_collection', '/pre_collection') for f in pre_file_list]
for i in range(len(pre_move_file_list)):
    shutil.move(pre_file_list[i],pre_move_file_list[i])

In [67]:
test_file_list=[f.replace('/row_collection', '/test_collection') for f in pre_file_list]
for i in range(len(pre_move_file_list)):
    ppg_count=0
    filename=os.path.splitext(os.path.basename(pre_move_file_list[i]))[0]
    df=pd.read_csv(pre_move_file_list[i])
    ppg=df['GREEN']
    if len(ppg)>3000:
        ppg_count=len(ppg)//3000
    for j in range(ppg_count):
        temp_ppg=ppg[j*3000:j*3000+3000]
        temp_ppg=temp_ppg-temp_ppg.min()
        temp_ppg=temp_ppg/temp_ppg.max()
        temp_ppg=temp_ppg
        temp_ppg.to_csv(os.path.dirname(test_file_list[0])+'/'+filename+'_'+str(j).zfill(4)+'.csv',index=False)

'../../data/test_collection/이창현'